# Overview

quickstart: https://python.langchain.com/docs/get_started/quickstart/

In [4]:
from langchain_openai import ChatOpenAI
from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')
openai_api_key = config['LangChain']['openai_api_key']
llm = ChatOpenAI(api_key=openai_api_key)

In [5]:
llm.invoke('how can langsmith help with testing?')

AIMessage(content='Langsmith can help with testing by providing a platform for creating and running automated tests for software applications. It can assist in generating test scripts, executing tests, analyzing test results, and reporting any issues found during testing. Langsmith can also help in creating test cases based on requirements and specifications, and can assist in managing test data and test environments. Overall, Langsmith can streamline the testing process and help improve the efficiency and effectiveness of testing activities.', response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 15, 'total_tokens': 104}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_d9767fc5b9', 'finish_reason': 'stop', 'logprobs': None}, id='run-e6b86f99-6c9d-4160-bc83-eeeb149bad6c-0')

## Prompts
Guide its response with a prompt template

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ('system', ' you are world class technical writer'),
    ('user','{input}')
])

Now combine these into a simple LLM chain

In [7]:
chain = prompt | llm

We can now invoke it and ask the same question. It still won't know the answer, but it should response with better tone.

In [8]:
chain.invoke({'input': 'how can langsmith help with testing?'})

AIMessage(content='Langsmith can help with testing in a variety of ways, thanks to its advanced language processing capabilities. Here are some ways in which Langsmith can be beneficial for testing:\n\n1. Automated Test Case Generation: Langsmith can be used to automatically generate test cases based on the requirements and specifications provided. By analyzing the language used in the requirements documents, Langsmith can create comprehensive test cases that cover all possible scenarios.\n\n2. Natural Language Testing: Langsmith can be used to perform natural language testing to verify that the software behaves as expected when interacting with users. This can help ensure that the software is user-friendly and that it responds appropriately to user inputs.\n\n3. Code Analysis: Langsmith can analyze code written in different programming languages to identify potential bugs, security vulnerabilities, or performance issues. This can help improve the quality of the software and reduce the

## Output Parser
However, it's often much more convenient to work with strings. Let's add a simple output parser to convert the chat message to a string.

In [9]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Now add this to the previous chain

In [11]:
chain = prompt | llm | output_parser

In [12]:
chain.invoke({'input': 'how can langsmith help with testing?'})

'Langsmith, a specialized testing tool, can greatly assist in the testing process by providing various features and functionalities that help streamline and optimize the testing efforts. Here are several ways in which Langsmith can help with testing:\n\n1. Test Automation: Langsmith offers automated testing capabilities that enable testers to automate the execution of test cases, saving time and effort in manual testing. This can help in running tests repeatedly and consistently, ensuring thorough test coverage.\n\n2. Test Case Management: Langsmith provides a centralized platform for managing test cases, test plans, and test results. Testers can create, organize, and prioritize test cases, as well as track the execution status and results of tests.\n\n3. Test Data Management: Langsmith facilitates the management of test data by providing tools for generating, importing, and manipulating test data sets. This helps in ensuring that tests are executed with relevant and accurate data.\n\n

# Model I/O
https://python.langchain.com/docs/modules/model_io/#prompt-templates

In [22]:
from langchain_openai import ChatOpenAI

# llm = OpenAI()
chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=openai_api_key)
llm = ChatOpenAI(api_key=openai_api_key)

In [23]:
from langchain_core.messages import HumanMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

print(f"LLM: {llm.invoke(text)}")

print(f"LLM: {chat_model.invoke(messages)}")

LLM: content='Rainbow Footwear' response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 22, 'total_tokens': 26}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None} id='run-1062dd94-9f0d-4f81-8dda-a7030e3b7b56-0'
LLM: content='RainbowFootwear' response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 22, 'total_tokens': 26}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_d9767fc5b9', 'finish_reason': 'stop', 'logprobs': None} id='run-679f0df8-fc5d-4c32-89a1-9bc7bd66c430-0'


## Prompt Templates
https://python.langchain.com/docs/modules/model_io/#prompt-templates

Most LLM applications do not pass user input directly into an LLM. Usually they will add the user input to a larger piece of text, called a prompt template, that provides additional context on the specific task at hand.

In [13]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template('what is a good name for a company that makes {product}?')
prompt.format(product='colorful socks')

'what is a good name for a company that makes colorful socks?'

In [14]:
from langchain.prompts.chat import ChatPromptTemplate

template = 'you are a helpful assistant that translates {input_language} to {output_language}.'
human_template = '{text}'

chat_prompt = ChatPromptTemplate.from_messages([
    ('system', template),
    ('human', human_template),
])

chat_prompt.format_messages(input_language='English', output_language='French', text='I love programming.')

[SystemMessage(content='you are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]

## Output parsers
https://python.langchain.com/docs/modules/model_io/#output-parsers

OutputParsers convert the raw output of a language model into a format that can be used downstream. There are a few main types of OutputParsers, including:

Convert text from LLM into structured information (e.g. JSON)
Convert a ChatMessage into just a string
Convert the extra information returned from a call besides the message (like OpenAI function invocation) into a string.

In [15]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
output_parser.parse('hi, bye')

['hi', 'bye']

## Composising with LCEL
We can now combine all these into one chain. This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to a language model, and then pass the output through an (optional) output parser.

In [27]:
template = "Generate a list of 5 {text}.\n\n{format_instructions}"

chat_prompt = ChatPromptTemplate.from_template(template)
chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions())

chain = chat_prompt | chat_model | output_parser
chain.invoke({'text':'colors'})

['red', 'blue', 'green', 'yellow', 'purple']